# Analysis

In [1]:
import numpy as np
import scipy
import math
import random
import matplotlib.pyplot as plt

In [2]:
from env import *
import utils

In [3]:
def observed_function_f(xs, a, b, c, d, frequency):
    results = []
    for x in xs:
        result = c * math.cos(a * x * frequency + b) + d
        results.append(result)
            
    return results

def observed_function_f1(xs, a, b, c, d):
    return observed_function_f(xs, a, b, c, d, 1)

def observed_function_f3(xs, a, b, c, d):
    return observed_function_f(xs, a, b, c, d, 3)

def observed_function_f5(xs, a, b, c, d):
    return observed_function_f(xs, a, b, c, d, 5)

def observed_function_f7(xs, a, b, c, d):
    return observed_function_f(xs, a, b, c, d, 7)

def observed_function_f9(xs, a, b, c, d):
    return observed_function_f(xs, a, b, c, d, 9)

## extraction

In [4]:
def plainQubitsExtraction(counts):
    return counts

def separateQubitsExtraction(counts):
    ##### calculating number of qubits on the device and shots done
    experiments_count = len(counts)
    qubits_count = None
    shots = 0
    one_experiment = counts[0]
    for key in one_experiment:
        shots += one_experiment[key]
        if qubits_count == None:
            qubits_count = len(key)
    
    ##### separating results for each qubit
    qs = [[0 for i in range(experiments_count)] for j in range(qubits_count)]

    ##### transforming results
    for i in range(experiments_count):
        counts_i = counts[i]
        for key in counts_i:
            j = 0
            for v in key:
                # |0> = 1, |1> = -1
                if v == "0":
                    qs[j][i] += counts_i[key]
                elif v == "1":
                    qs[j][i] -= counts_i[key]
                        
                j += 1
        
    ##### from qiskit arrangment to physical
    qs = qs[::-1]
    qs = np.array(qs) / shots
    
    return qs

### process 

In [5]:
# NB! experiments should be sorted by step offset
# e.g. for step == 0.1 lets say
#      first batch contains results for parameter values 1, 2, 3
#      next - for 1.1, 2.1, 3.1
#      next - for 1.2, 2.2, 3.2
#      ...

def combine(experiments, parameter):
    all_counts = []
    all_parameter_values = []
    for experiment_name in experiments:
        counts, parameter_values = processParametrizedExperiment(experiment_name,
                                                                 THETA,
                                                                 plainQubitsExtraction)
        all_counts.append(counts)
        all_parameter_values.append(parameter_values)
    
    combined_counts = []
    combined_parameter_values = []
    
    experiments_count = len(all_counts)
    batch_count = len(all_counts[0])
    for result_index in range(batch_count):
        for batch_index in range(experiments_count):
            combined_counts.append(all_counts[batch_index][result_index])
            combined_parameter_values.append(all_parameter_values[batch_index][result_index])
    
    return combined_counts, combined_parameter_values

def processCombinedExperiment(experiment_names, parameter):
    counts, parameter_values = combine(experiment_names, parameter)
    qs = separateQubitsExtraction(counts)
    return qs, parameter_values

In [6]:
def rebuildCounts(counts, desired_shots):
    measurements = []
    for value in counts:
        observations = counts[value]
        for i in range(observations):
            measurements.append(value)

    new_measurements = random.sample(measurements, desired_shots)
    new_counts = {}
    for value in counts:
        new_counts[value] = new_measurements.count(value)
    
    return new_counts
    
def processParametrizedExperiment(experiment_name, parameter, qubitsExtraction, shots = None):
    path = "../experiments/" + experiment_name + ".json"
    json_object = utils.retrieve(path)
    
    ##### retrieving experiment from the file
    parameters = json_object.get("parameters")
    parameter_values = parameters.get(str(parameter))
    counts = json_object.get("counts")
    
    experiments_count = len(parameter_values) # == len(counts)
    if counts == None:
        ### retrive job from backend
        account_id = json_object["account"]
        device_id = json_object["backend"]["name"]
        device = qutils.backend(account_id, device_id)
        jobId = json_object["job"]
        job = device.retrieve_job(jobId)
        error_message = job.error_message()
        if error_message:
            print("ERROR: " + error_message, "\nEXPERIMENT: " + experiment_name)
            
        result = job.result()
        counts = []
        for i in range(experiments_count):
            i_counts = result.get_counts(i)
            counts.append(i_counts)
                
        ### saving results locally
        utils.update(json_object, counts, path)
    
    if not shots == None:
        for i in range(len(counts)):
            i_counts = counts[i]
            counts[i] = rebuildCounts(i_counts, shots)
    
    return qubitsExtraction(counts), parameter_values

def processExperiment(experiment_name):
    path = "experiments/" + experiment_name + ".json"
    json_object = utils.retrieve(path)
    
    ##### retrieving experiment from the file
    counts = json_object.get("counts")
    
    if counts == None:
        ### retrive job from backend
        account_id = json_object["account"]
        device_id = json_object["backend"]["name"]
        device = qutils.backend(account_id, device_id)
        jobId = json_object["job"]
        job = device.retrieve_job(jobId)
        error_message = job.error_message()
        if error_message:
            print("ERROR: " + error_message, "\nEXPERIMENT: " + experiment_name)
            
        result = job.result()
        counts = result.get_counts()
        ### saving results locally
        utils.update(json_object, counts, path)
    
    return counts

def processParametrizedExperimentWithRealMeasurement(experiment_name, parameter, account_for_):
    counts, parameter_values = processParametrizedExperiment(experiment_name, parameter, plainQubitsExtraction)
    return utils.real_measurement(counts, account_for_)

### analyze

In [7]:
def mean_square_error(A, B):
    return np.square(np.subtract(A, B)).mean()

def analyzeExperiment(experiment_name, observed_function, shots = None):
    counts, parameter_values = processParametrizedExperiment(experiment_name,
                                                             THETA,
                                                             separateQubitsExtraction,
                                                             shots)
    target_qubit_results = counts[0]
    xdata = parameter_values
    ydata = target_qubit_results

    
    fit_result = scipy.optimize.curve_fit(observed_function, xdata, ydata)
    fitted_params = fit_result[0]
    sim_results = observed_function(xdata, fitted_params[0], fitted_params[1], fitted_params[2], fitted_params[3])
    return parameter_values, target_qubit_results, sim_results, fitted_params

def analyzeExperiments(experiments_names, observed_functions, shots = None):
    for i in range(len(experiments_names)):
        experiment_name = experiments_names[i]
        observed_function = observed_functions[i]
        parameter_values, target_qubit_results, sim_results, fitted_params = analyzeExperiment(experiment_name,
                                                                                               observed_function,
                                                                                               shots)
        
        fitted_function = str(fitted_params[2]) + ' * cos(' + str(fitted_params[0]) \
            + ' * x + ' + str(fitted_params[1]) + ') + ' + str(fitted_params[3])
        mse = mean_square_error(target_qubit_results, sim_results)
        
        path_to_save_fig = None#"../experiments/" + a_experiment_name + "_fitted.pdf"
        utils.plot([parameter_values, parameter_values],
                   [target_qubit_results, sim_results],
                   curves_names = ['measured', fitted_function],
                   title = 'Mean Square Error: ' + str(mse),
                   x_name = 'parameter value',
                   y_name = 'expectation',
                   path_to_file = path_to_save_fig,
                   include_ft = False)

In [8]:
def resultsOfExperiments(experiments_names):
    for a_experiment_name in experiments_names:
        a_qs, a_parameter_values = processParametrizedExperiment(a_experiment_name,
                                                                 THETA,
                                                                 separateQubitsExtraction)
        path_to_save_fig = "../experiments/" + a_experiment_name + "_result.pdf"
        plot([a_parameter_values for i in range(len(a_qs))], a_qs)

### maximum likelyhood estimation

In [ ]:
# ML

# theta_a - rotation
# m_k - iterations of fusion
# h_k - good outcomes
# N_k - all the outcomes

def L_k(theta_a, m_k, h_k, N_k):
    angle = (2 * m_k + 1) * theta_a
    sin_2 = pow(math.sin(angle), 2)
    cos_2 = pow(math.cos(angle), 2)
    result = pow(sin_2, h_k) * pow(cos_2, N_k - h_k)
    return result

def L(theta_a, m_ks, h_ks, N_ks):
    result = 1
    for i in range(len(m_ks)):
        m_k = m_ks[i]
        h_k = h_ks[i]
        N_k = N_ks[i]
        result *= L_k(theta_a, m_k, h_k, N_k)
    
    return result

def MLE_L(param):
    global global_m_ks, global_h_ks, global_N_ks
    value = L(param, global_m_ks, global_h_ks, global_N_ks)
    return -value

def ln_L_k(theta_a, m_k, h_k, N_k):
    angle = (2 * m_k + 1) * theta_a
    sin_2 = pow(math.sin(angle), 2)
    cos_2 = pow(math.cos(angle), 2)
    
    # we can do the following, because log(x) ~ log(x + "a little")
    a_little = 1e-323
    sin_2 += a_little
    cos_2 += a_little
    
    result = h_k * math.log(sin_2) + (N_k - h_k) * math.log(cos_2)
    return result

def ln_L(theta_a, m_ks, h_ks, N_ks):
    result = 0
    for i in range(len(m_ks)):
        m_k = m_ks[i]
        h_k = h_ks[i]
        N_k = N_ks[i]
        result += ln_L_k(theta_a, m_k, h_k, N_k)
    
    return result

def MLE_ln_L(param):
    global global_m_ks, global_h_ks, global_N_ks
    value = ln_L(param, global_m_ks, global_h_ks, global_N_ks)
    return -value


a_experiments = #array of experiment names#
a_range = range(len(a_experiments))
a_target_qubit_index = 0
theta_indeces = 75

global_m_ks = [i for i in a_range]
global_N_ks = [100 for i in a_range]
MLE_results = []

a_steps = 100
a_step = math.pi * 2 / a_steps
MLE_points = []
gammas = [a_step * step for step in range(a_steps)]

plt_gammas = []
plt_thetas = []
plt_MLE_ln_Ls = []

for a_theta_index in range(theta_indeces):
    global_h_ks = []
    for i in a_range:
        a_experiment_name = a_experiments[i]
        a_shots = global_N_ks[i]

        a_qs, a_parameter_values = processParametrizedExperiment(a_experiment_name,
                                                                 THETA,
                                                                 plainQubitsExtraction,
                                                                 a_shots)
        
        h_k = 0
        a_experiment_results = a_qs[a_theta_index]
        for outcome in a_experiment_results:
            if outcome[a_target_qubit_index] == '1':
                h_k += a_experiment_results[outcome]

        global_h_ks.append(h_k)

    plt_thetas.extend([a_parameter_values[a_theta_index] for i in range(a_steps)])
    plt_gammas.extend(gammas)
    plt_MLE_ln_Ls.extend([MLE_ln_L(a_gamma) for a_gamma in gammas])
    
    min_f = None
    min_gamma = None

    for a_gamma in gammas:
        current_min_gamma = scipy.optimize.fminbound(MLE_ln_L, a_gamma, a_gamma + a_step)
        current_min_f = MLE_ln_L(current_min_gamma)
        if min_f == None or min_f > current_min_f:
            min_gamma = current_min_gamma
            min_f = current_min_f
    
    optimized_result = pow(math.cos(min_gamma), 2) - pow(math.sin(min_gamma), 2)
    MLE_results.append(optimized_result)

xs = a_parameter_values
ys = MLE_results

plt.plot(xs, ys)
plt.show()

In [ ]:
for a_theta_index in range(theta_indeces):
    left_i = a_theta_index * a_steps
    right_i = (a_theta_index + 1) * a_steps
    xs = plt_gammas[left_i:right_i]
    ys = plt_MLE_ln_Ls[left_i:right_i]

    plt.plot(xs, ys)

    plt.show()